In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
r = requests.get('http://books.toscrape.com')
soup = BeautifulSoup(r.text,'html.parser')
cat = soup.find_all("ul",attrs={"class":"nav nav-list"})
cat = str(cat[0])
soup = BeautifulSoup(cat,'html.parser')
cat = soup.find_all("li")
categorias_resultado = list()
for categorias in cat:
    site = 'http://books.toscrape.com/'+categorias.a['href']
    categorias_resultado.append([site,categorias.a.text.strip()])
categorias_resultado = categorias_resultado[1:]

In [5]:
data_set = list()
for link, cate in categorias_resultado:
    r = requests.get(link)
    soup = BeautifulSoup(r.text,'html.parser')
    livros = soup.find_all("li",attrs={"class":"col-xs-6 col-sm-4 col-md-3 col-lg-3"})
    for data in livros:
        nome = data.h3.a['title']
        preco = data.findAll('p',attrs={"class":"price_color"})[0].text.replace('Â£','')
        estrela = data.p['class'][1]
        estoque = data.findAll('p',attrs={"class":"instock availability"})[0].text.strip()
        data_set.append([nome,preco,cate,estrela,estoque])

In [21]:
import pandas as pd
df = pd.DataFrame(data_set,columns=['Nome','Preco','Categoria','Estrelas','Estoque'])

In [19]:
df.Estrelas.unique()

array(['Two', 'Four', 'Three', 'One', 'Five'], dtype=object)

In [24]:
df.loc[df.Estrelas=='Two','Estrelas'] = 2
df.loc[df.Estrelas=='Four','Estrelas'] = 4
df.loc[df.Estrelas=='Three','Estrelas'] = 3
df.loc[df.Estrelas=='One','Estrelas'] = 1
df.loc[df.Estrelas=='Five','Estrelas'] = 5

In [26]:
df.Estoque.unique()

array(['In stock'], dtype=object)

In [27]:
df.loc[df.Estoque=='In stock','Estoque'] = 1

In [28]:
df

,Nome,Preco,Categoria,Estrelas,Estoque
0,It's Only the Himalayas,45.17,Travel,2,1
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,Travel,4,1
2,See America: A Celebration of Our National Par...,48.87,Travel,3,1
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,Travel,2,1
4,Under the Tuscan Sun,37.33,Travel,3,1
...,...,...,...,...,...
512,Why the Right Went Wrong: Conservatism--From G...,52.65,Politics,4,1
513,Equal Is Unfair: America's Misguided Fight Aga...,56.86,Politics,1,1
514,Amid the Chaos,36.58,Cultural,1,1
515,Dark Notes,19.19,Erotica,5,1


In [30]:
df.to_csv('livros_data_set.csv')